In [33]:
import os
import cv2
from PIL import Image
import numpy as np
import pickle as pkl
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from processer import equalize, zmIceColor, extractGreen
from dataset import PlantSeedDataset
datasets_path = "..\\dataset\\classifer\\train"
    
transform = transforms.Compose([
        transforms.Resize((256, 256)),
        np.asarray,
        transforms.Lambda(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR))
        ])
dataset = PlantSeedDataset(datasets_path, transform=transform)
dataloader = DataLoader(dataset)



In [81]:
for img, label, img_name in dataset:
    print(type(img), type(label), type(img_name))
    break

<class 'numpy.ndarray'> <class 'str'> <class 'str'>


In [56]:
import pickle as pkl
gre_imgs = []
sift_features = []
for img, label, img_name in dataset:
    gre_img = processer(img)
    kp, sift_feature = SIFT(gre_img)
    gre_imgs.append(gre_img)
    sift_features.append(sift_feature)

with open("sift_features.pkl", "wb") as f:  # 序列化
    pkl.dump(sift_features, f) 
with open("gre_img.pkl", "wb") as f:
    pkl.dump(gre_imgs, f)
len(sift_features)

4440

In [62]:
with open('gre_img.pkl', 'rb') as f:
    gre_imgs = pkl.load(f)
sift_features = []
for img in gre_imgs:
    kp, sift_feature = SIFT(img)
    sift_features.append(sift_feature)
len(sift_features), len(gre_imgs)

4440

In [3]:
import pickle as pkl
with open('gre_img.pkl', 'rb') as f:
    gre_imgs = pkl.load(f)
with open('sift_features.pkl', 'rb') as f:
    sift_features = pkl.load(f)

In [4]:
from processer import bow_feature, bow_init
bow_extractor = bow_init(sift_features)
all_feature_bow = bow_feature(bow_extractor, gre_imgs)

In [5]:
len(all_feature_bow)

4440

In [6]:
with open("all_feature_bow.pkl", "wb") as f:  # 序列化
    pkl.dump(all_feature_bow, f) 

In [7]:
all_feature_bow[0].shape

(1, 100)

In [6]:
import pickle as pkl
with open('gre_img.pkl', 'rb') as f:
    gre_imgs = pkl.load(f)
hog_features = []
for img in gre_imgs:
    hog_feature, hog_img = HOG(img)
    hog_features.append(hog_feature)
with open("hog_features.pkl", "wb") as f:  # 序列化
    pkl.dump(hog_features, f) 

In [29]:
with open('hog_features.pkl', 'rb') as f:
    hog_features = pkl.load(f)

In [2]:
hog_features[0].shape

(129600,)

In [28]:
feature_lbps = []
for img in gre_imgs:
    feature_lbp = LBP(img, n_points=2, radius=2)
    feature_lbps.append(feature_lbp)
with open("feature_lbps22.pkl", "wb") as f:  # 序列化  后两位为参数 不加的为默认参数
    pkl.dump(feature_lbps, f) 

In [4]:
# feature_lbps[0].shape
# cv2.imwrite("lbp.png", feature_lbps[0])
with open('gre_img.pkl', 'rb') as f:
    gre_imgs = pkl.load(f)

True

In [27]:
i = 265
org_img = gre_imgs[i]
cv2.imwrite(f"org_img_{i}.png", org_img)
lbp_img = LBP(org_img, n_points=2, radius=2, method='var')
cv2.imwrite(f"lbp_{i}.png", lbp_img)

True

In [37]:
i = 0
org_img = gre_imgs[i]
cv2.imwrite(f"org_img_{i}.png", org_img)
hog_feature, hog_img = HOG(org_img, orientations=9, pixels_per_cell=8, cells_per_block=3)
# hog_feature, hog_img = HOG(org_img)
cv2.imwrite(f"hog_{i}.png", hog_img)
hog_feature.shape

(72900,)

In [38]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3)  # t分布随机邻居嵌入 降维
embeddings = tsne.fit_transform(np.asarray(hog_features[:200]))

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

In [57]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components = 100, kernel="rbf", gamma=0.04) 
hog_pca = rbf_pca.fit_transform(np.asarray(hog_features)) 

In [88]:
hog_pca.shape

(4440, 100)

In [89]:
np.asarray(hog_features).shape

(4440, 129600)

In [59]:
np.save('hog_pca.npy', hog_pca)

In [60]:
with open("feature_lbps22.pkl", "rb") as f:  # 序列化  后两位为参数 不加的为默认参数
    lbp_features = pkl.load(f)

In [69]:
for i in range(len(lbp_features)):
    lbp_features[i] = lbp_features[i].reshape(256*256, 3)

In [74]:
np.asarray(lbp_features).shape

(4440, 65536, 3)

In [92]:
for i in range(len(lbp_features)):
    lbp_features[i] = lbp_features[i].reshape(3, -1)

In [93]:
lbp_features[0].shape

(3, 65536)

In [73]:
np.transpose(lbp_features[0]).shape

(3, 65536)

In [94]:
import pandas as pd
lbp_fill = []
for lbp_feature in lbp_features:
    feature_df = pd.DataFrame(lbp_feature)
    lbp_fill.append(feature_df.fillna(0))  # 将缺失值部分填充0

In [95]:
lbp_fill[0].shape

(3, 65536)

In [97]:
lbp_pca = []
for lbp_feature in lbp_fill:
    rbf = KernelPCA(n_components = 1000, kernel="rbf", gamma=0.04)
    lbp_pca1 = rbf.fit_transform(lbp_feature)
    print(lbp_pca1.shape)
    break
    rbf_pca = KernelPCA(n_components = 100, kernel="rbf", gamma=0.04) 
    lbp_pca.append(rbf_pca.fit_transform(np.transpose(lbp_feature)))
# lbp_pca = np.asarray(lbp_pca)
# np.save('lbp_pca.npy', lbp_pca)

(3, 3)


In [98]:
lbp_fill_2d = np.asarray(lbp_fill).reshape(-1, 65536)

In [99]:
lbp_fill_2d.shape

(13320, 65536)

In [100]:
rbf = KernelPCA(n_components = 100, kernel="rbf", gamma=0.04)
lbp_pca = rbf.fit_transform(lbp_fill_2d)

In [101]:
lbp_pca.shape

(13320, 100)

In [103]:
lbp_pca = lbp_pca.reshape(-1, 3, 100)

In [104]:
lbp_pca.shape

(4440, 3, 100)

In [105]:
np.save('lbp_pca.npy', lbp_pca)

In [106]:
from xgboost import XGBClassifier
import xgboost
from sklearn.model_selection import train_test_split

In [2]:
import pickle as pkl
with open("all_feature_bow.pkl", "rb") as f:  # 序列化
    all_feature_bow = pkl.load(f) 

EOFError: Ran out of input